#### Simple GenAI APP using Langchain

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
## Langsmith Tracking
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [18]:
#data ingestion --from website we need to scrape the data
from langchain_community.document_loaders import WebBaseLoader

loader=WebBaseLoader("https://www.langchain.com/pricing-langsmith?_gl=1*91275m*_gcl_au*NDYzNjI2NTkyLjE3NTExMzk1MTA.*_ga*NzM2NjExNTExLjE3NTEwNTg2NzI.*_ga_47WX3HKKY2*czE3NTExMzk1MTAkbzQkZzEkdDE3NTExMzk1NDQkajI2JGwwJGgw")
loader

In [19]:
docs=loader.load()
docs

[Document(metadata={'source': 'https://www.langchain.com/pricing-langsmith?_gl=1*91275m*_gcl_au*NDYzNjI2NTkyLjE3NTExMzk1MTA.*_ga*NzM2NjExNTExLjE3NTEwNTg2NzI.*_ga_47WX3HKKY2*czE3NTExMzk1MTAkbzQkZzEkdDE3NTExMzk1NDQkajI2JGwwJGgw', 'title': 'LangSmith Pricing', 'description': "LangSmith pricing for teams of any size. Choose the plan that suits your needs, whether you're an individual developer or enterprise. Debug, test, and monitor your LLM apps confidently.", 'language': 'en'}, page_content='LangSmith Pricing\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nProducts\n\nFrameworksLangGraphLangChainPlatformsLangSmithLangGraph PlatformResources\n\nResources HubBlogCustomer StoriesLangChain AcademyCommunityExpertsChangelogDocs\n\nPythonLangGraphLangSmithLangChainJavaScriptLangGraphLangSmithLangChainCompany\n\nAboutCareersPricing\n\nLangSmithLangGraph PlatformGet a demoSign upLangSmith plans for teams of\xa0any\xa0sizeLooking for LangGraph Platform pricing?StartupsDesigned for early stage startup

In [20]:
### Load Data--> Docs-->Divide our Docuemnts into chunks documents
# text-->vectors-->Vector Embeddings
# --->Vector Store DB


from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
documents=text_splitter.split_documents(docs)


In [21]:
documents

[Document(metadata={'source': 'https://www.langchain.com/pricing-langsmith?_gl=1*91275m*_gcl_au*NDYzNjI2NTkyLjE3NTExMzk1MTA.*_ga*NzM2NjExNTExLjE3NTEwNTg2NzI.*_ga_47WX3HKKY2*czE3NTExMzk1MTAkbzQkZzEkdDE3NTExMzk1NDQkajI2JGwwJGgw', 'title': 'LangSmith Pricing', 'description': "LangSmith pricing for teams of any size. Choose the plan that suits your needs, whether you're an individual developer or enterprise. Debug, test, and monitor your LLM apps confidently.", 'language': 'en'}, page_content='LangSmith Pricing\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nProducts\n\nFrameworksLangGraphLangChainPlatformsLangSmithLangGraph PlatformResources\n\nResources HubBlogCustomer StoriesLangChain AcademyCommunityExpertsChangelogDocs\n\nPythonLangGraphLangSmithLangChainJavaScriptLangGraphLangSmithLangChainCompany\n\nAboutCareersPricing'),
 Document(metadata={'source': 'https://www.langchain.com/pricing-langsmith?_gl=1*91275m*_gcl_au*NDYzNjI2NTkyLjE3NTExMzk1MTA.*_ga*NzM2NjExNTExLjE3NTEwNTg2NzI.*_ga_47WX

In [22]:
from langchain_openai import OpenAIEmbeddings
embeddings=OpenAIEmbeddings()

In [23]:
from langchain_community.vectorstores import FAISS
vectorstoredb=FAISS.from_documents(documents,embeddings)
vectorstoredb

In [ ]:
## Query From a vector db
query="LangSmith plans for teams of any size"
result=vectorstoredb.similarity_search(query)
result[0].page_content

'LangSmithLangGraph PlatformGet a demoSign upLangSmith plans for teams of\xa0any\xa0sizeLooking for LangGraph Platform pricing?StartupsDesigned for early stage startups building AI applicationsReach out for starter pricing and get shipping todayReach outWhat to expect:We want all early stage companies to build with LangSmith. LangSmith for Startups offers discounted prices and a generous free, monthly trace allotment, so you can have the right tooling in place as you grow your'

In [30]:
#till now we imported data ,divided into chunks,converted into vectors ,then stored in vectordb 
# That lets us find similar chunks for a given query — but to get meaningful answers, we need to pass those chunks to a language model. That’s where RAG (Retrieval-Augmented Generation) comes in.

In [31]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="gpt-4o")

In [ ]:
#Retrival Chain,Documents Chain 
#A document chain in LangChain is any chain designed to:Take a list of documents and process them in a way that’s useful for an LLM.

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context:
<context>
{context}
</context>


"""
)

document_chain=create_stuff_documents_chain(llm,prompt)
document_chain


RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n\n'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x132759310>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x1327591d0>, root_client=<openai.OpenAI object at 0x1325bad70>, root_async_client=<openai.AsyncOpenAI object at 0x1325baea0>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_docume

In [35]:
from langchain_core.documents import Document
document_chain.invoke({
    "input":"LangSmith plans for teams of any size",
    "context":[Document(page_content="LangSmith plans for teams of any size Looking for LangGraph Platform pricing?")]
})

"The provided context does not offer specific information about LangGraph Platform pricing. It simply mentions that LangSmith plans for teams of any size but doesn't provide details about the pricing of the LangGraph Platform."

However, we want the documents to first come from the retriever we just set up. That way, we can use the retriever to dynamically select the most relevant documents and pass those in for a given question.

In [ ]:
### input--->retriever--->vectorstoredb
retriever=vectorstoredb.as_retriever()



In [39]:
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)

In [40]:
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x1165c20d0>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n\n'), additional_kwargs={})])
            | ChatOpe

In [43]:
#GET THE RESPONSE FROM THE LLM
response=retrieval_chain.invoke({"input":"LangSmith plans for teams of any size"})
response['answer']

'The LangSmith platform offers various plans tailored to different needs. For startups, there is the "Startup Plan" which provides discounted pricing and generous free monthly trace allotments. This plan is designed to support early-stage companies in building AI applications, allowing them to establish the proper tooling as they grow. Startups can stay on this plan for up to 2 years before transitioning to the Plus Plan. Additionally, the Developer plan is targeted at personal projects with 1 free seat and 5k base traces per month included. The Plus plan allows for up to 10 seats with 10k base traces per month, and the Enterprise plan caters to teams with potentially larger-scale needs.'

In [44]:
response['context']

[Document(id='8d6569b7-10ac-4bbe-9812-e3870196ec5c', metadata={'source': 'https://www.langchain.com/pricing-langsmith?_gl=1*91275m*_gcl_au*NDYzNjI2NTkyLjE3NTExMzk1MTA.*_ga*NzM2NjExNTExLjE3NTEwNTg2NzI.*_ga_47WX3HKKY2*czE3NTExMzk1MTAkbzQkZzEkdDE3NTExMzk1NDQkajI2JGwwJGgw', 'title': 'LangSmith Pricing', 'description': "LangSmith pricing for teams of any size. Choose the plan that suits your needs, whether you're an individual developer or enterprise. Debug, test, and monitor your LLM apps confidently.", 'language': 'en'}, page_content='LangSmithLangGraph PlatformGet a demoSign upLangSmith plans for teams of\xa0any\xa0sizeLooking for LangGraph Platform pricing?StartupsDesigned for early stage startups building AI applicationsReach out for starter pricing and get shipping todayReach outWhat to expect:We want all early stage companies to build with LangSmith. LangSmith for Startups offers discounted prices and a generous free, monthly trace allotment, so you can have the right tooling in plac